In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.9.2


In [5]:
train_dir2="/content/drive/MyDrive/cbis_ddsm/mass/train"
test_dir="/content/drive/MyDrive/cbis_ddsm/mass/test"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_dir="/content/drive/MyDrive/BreaKHis_v1/histology_slides/breast"

In [22]:
train_data_7k = tf.keras.preprocessing.image_dataset_from_directory(train_dir, color_mode="grayscale",image_size=(227,227), batch_size=128)


Found 7909 files belonging to 2 classes.


In [38]:
train_data_7k_256 = tf.keras.preprocessing.image_dataset_from_directory(train_dir, color_mode="grayscale",image_size=(227,227), batch_size=256)


Found 7909 files belonging to 2 classes.


In [15]:
train_data2 = tf.keras.preprocessing.image_dataset_from_directory(train_dir2, color_mode="grayscale",image_size=(227,227), batch_size=10)

Found 552 files belonging to 3 classes.


In [32]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(227, 227, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Print the model summary
model.summary()

In [33]:
def FCN_model(len_classes=3,shape=(227, 227, 1), dropout_rate=0.2):
    
    input = tf.keras.layers.Input(shape=shape)

    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1,padding="valid")(input)
    x= tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.MaxPooling2D(2)(x)

    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.MaxPooling2D(2)(x)

    x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.MaxPooling2D(2)(x)

    x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.MaxPooling2D(2)(x)

    x = tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    
    # Fully connected layer 1
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Fully connected layer 2
    x = tf.keras.layers.Conv2D(filters=len_classes, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    predictions = tf.keras.layers.Activation('softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=predictions)
    
    print(model.summary())
    print(f'Total number of layers: {len(model.layers)}')

    return model

In [34]:
fcn0=FCN_model()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 227, 227, 1)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 225, 225, 32)      320       
                                                                 
 dropout_3 (Dropout)         (None, 225, 225, 32)      0         
                                                                 
 batch_normalization (BatchN  (None, 225, 225, 32)     128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 225, 225, 32)      0         
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                        

In [42]:
fcn1=FCN_model()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 227, 227, 1)]     0         
                                                                 
 conv2d_26 (Conv2D)          (None, 225, 225, 32)      320       
                                                                 
 dropout_17 (Dropout)        (None, 225, 225, 32)      0         
                                                                 
 batch_normalization_14 (Bat  (None, 225, 225, 32)     128       
 chNormalization)                                                
                                                                 
 activation_14 (Activation)  (None, 225, 225, 32)      0         
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                      

In [24]:
Adam0=tf.keras.optimizers.Adam(learning_rate=0.001)

In [29]:
model.compile(optimizer=Adam0,#RMSprop(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [35]:
fcn0.compile(optimizer=Adam0,#RMSprop(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [43]:
fcn1.compile(optimizer=Adam0,#RMSprop(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [30]:
history = model.fit(train_data_7k,epochs=5)

Epoch 1/5
62/62 [==============================] - 892s 13s/step - loss: 4.6415 - accuracy: 0.6144
Epoch 2/5
62/62 [==============================] - 96s 1s/step - loss: 0.6298 - accuracy: 0.6864
Epoch 3/5
62/62 [==============================] - 91s 1s/step - loss: 0.6275 - accuracy: 0.6864
Epoch 4/5
62/62 [==============================] - 92s 1s/step - loss: 0.6244 - accuracy: 0.6883
Epoch 5/5
62/62 [==============================] - 91s 1s/step - loss: 0.6149 - accuracy: 0.6950


In [36]:
history2 = fcn0.fit(train_data_7k,epochs=5)

Epoch 1/5
62/62 [==============================] - 98s 1s/step - loss: 1.0848 - accuracy: 0.4969
Epoch 2/5
62/62 [==============================] - 96s 1s/step - loss: 0.9387 - accuracy: 0.6276
Epoch 3/5
62/62 [==============================] - 95s 1s/step - loss: 0.8642 - accuracy: 0.6701
Epoch 4/5
62/62 [==============================] - 97s 1s/step - loss: 0.8039 - accuracy: 0.7098
Epoch 5/5
62/62 [==============================] - 93s 1s/step - loss: 0.7764 - accuracy: 0.7053


In [44]:
history3 = fcn1.fit(train_data_7k_256,epochs=20)

Epoch 1/20
31/31 [==============================] - 97s 2s/step - loss: 1.1124 - accuracy: 0.5370
Epoch 2/20
31/31 [==============================] - 100s 2s/step - loss: 0.9561 - accuracy: 0.6397
Epoch 3/20
31/31 [==============================] - 99s 2s/step - loss: 0.8752 - accuracy: 0.6771
Epoch 4/20
31/31 [==============================] - 98s 2s/step - loss: 0.8231 - accuracy: 0.6915
Epoch 5/20
31/31 [==============================] - 98s 2s/step - loss: 0.7998 - accuracy: 0.6965
Epoch 6/20
31/31 [==============================] - 101s 2s/step - loss: 0.7744 - accuracy: 0.7016
Epoch 7/20
31/31 [==============================] - 99s 2s/step - loss: 0.7447 - accuracy: 0.7132
Epoch 8/20
31/31 [==============================] - 100s 2s/step - loss: 0.7286 - accuracy: 0.7121
Epoch 9/20
31/31 [==============================] - 101s 2s/step - loss: 0.7003 - accuracy: 0.7247
Epoch 10/20
31/31 [==============================] - 97s 2s/step - loss: 0.6842 - accuracy: 0.7277
Epoch 11/20
31/